# Post-processing of case studies

In [1]:
import os
import cases

import matplotlib.pyplot as plt

from buildingspy.io.outputfile import Reader
from buildingspy.io.postprocess import Plotter

In [2]:
plt.rcParams['axes.facecolor']='whitesmoke'
plt.rcParams['font.size'] = 6
plt.rcParams['text.usetex'] = False
plt.rcParams['legend.facecolor'] = 'white'
plt.rcParams['legend.framealpha'] = 0.75
plt.rcParams['legend.edgecolor'] = 'none'

def save_plot(figure, file_name):
    """ Save the figure to a pdf and png file
    """
    figure.savefig('{}.pdf'.format(file_name))
    figure.savefig('{}.png'.format(file_name))
    

def configure_axes(axes):
    """ Configure the axis style
    """
    axes.spines['right'].set_visible(False)
    axes.spines['top'].set_visible(False)
    axes.spines['left'].set_visible(False)
    axes.spines['bottom'].set_visible(False)
    axes.grid(color='grey', linewidth=0.25)
    return

## Read data

In [3]:
winter=0
spring=1
summer=2
r_base=list()
r_base.insert(winter, Reader(cases.get_result_file_name('winter_base'), "dymola"))
r_base.insert(spring, Reader(cases.get_result_file_name('spring_base'), "dymola"))
r_base.insert(summer, Reader(cases.get_result_file_name('summer_base'), "dymola"))
print("Read base short simulations")

r_g36=list()
r_g36.insert(winter, Reader(cases.get_result_file_name('winter_g36'), "dymola"))
r_g36.insert(spring, Reader(cases.get_result_file_name('spring_g36'), "dymola"))
r_g36.insert(summer, Reader(cases.get_result_file_name('summer_g36'), "dymola"))
print("Read G36 short simulations")

Read base short simulations
Read G36 short simulations


## Room temperatures

In [4]:
def plot_room_temp(readers):
    ''' Main method that plots the results
    '''
    # ------------------------------------------------------
    from buildingspy.io.outputfile import Reader
    import matplotlib.pyplot as plt
    import matplotlib

    # Optionally, change fonts to use LaTeX fonts
    #from matplotlib import rc
    #rc('text', usetex=True)
    #rc('font', family='serif')

    room_styles = ['k', 'b', 'g', 'r', 'c', 'g']
    room_names = ['nor', 'wes', 'sou', 'eas', 'cor']  
    plt.clf()   
    time_scale=86400.
    # ------------------------------------------------------

    # Read results
    for i in [0, 1, 2]:
        (t, TOut) = readers[i].values("weaBus.TDryBul")
        t = t/time_scale
        
        TRoo = list()
        reader=readers[i]
        
        for iRoo in range(len(room_names)):
            TRoo.insert(iRoo, reader.values(\
                "flo.{}.air.vol.T".format(room_names[iRoo]))[1])
    
        TSetHea = reader.values("conVAVCor.TRooHeaSet")[1]
        TSetCoo = reader.values("conVAVCor.TRooCooSet")[1]
   
        # Plot figure
        #plt.subplot(2, 1, 1)
        ax = plt.subplot(3, 1, i+1)

        ax.plot(t, TOut-273.15,    'gray', label='$T_{out}$', linewidth=0.25)
        for iRoo in range(len(room_names)):
            ax.plot(t, TRoo[iRoo]-273.15, room_styles[iRoo], \
                    label='$T_{{{name}}}$'.format(name=room_names[iRoo]), linewidth=0.25)
    
        ax.patch.set_facecolor('mistyrose')    
        ax.fill_between(t, TSetHea-273.15, y2=TSetCoo-273.15, color='whitesmoke')
    
        ax.set_xlabel('time [days]')
        ax.set_ylabel('temperature [$^\circ$C]')
    #ax.xaxis.set_ticks(np.arange(min(t)+0, 365, 1))

        ax.set_xlim([min(t)+5, min(t)+12])
                
        ax.legend(loc='center right', ncol=1)

        configure_axes(ax)

    return plt

# Create the plots
fig = plot_room_temp(r_base)
save_plot(fig, "TRoom_base")
fig = plot_room_temp(r_g36)
save_plot(fig, "TRoom_g36")

In [5]:
def plot_vav(readers):
    ''' Main method that plots the results
    '''
    # ------------------------------------------------------
    from buildingspy.io.outputfile import Reader
    import matplotlib.pyplot as plt

    room_styles = ['k', 'b', 'g', 'r', 'c', 'g']
    room_names = ['nor', 'wes', 'sou', 'eas', 'cor']  
    plt.clf()   
    time_scale=86400.
    # ------------------------------------------------------

    # Read results
    for i in [0, 1, 2]:
        (t, TOut) = readers[i].values("weaBus.TDryBul")
        t = t/time_scale
        
        yHea = list()
        yVAV = list()
        
        reader=readers[i]
        
        for kRoo in range(len(room_names)):
            yHea.insert(kRoo, reader.values(\
                "{}.yVal".format(room_names[kRoo]))[1])
            yVAV.insert(kRoo, reader.values(\
                "{}.yVAV".format(room_names[kRoo]))[1])
            
        # Plot figure
        #plt.subplot(2, 1, 1)
        ax = plt.subplot(3, 1, i+1)

        for iRoo in [0, 2]:
            ax.plot(t, yHea[iRoo], room_styles[iRoo], \
                   label='$y_{{hea,{name}}}$'.format(name=room_names[iRoo]), linewidth=0.25)
            ax.plot(t, yVAV[iRoo], room_styles[iRoo+1], \
                    label='$y_{{vav,{name}}}$'.format(name=room_names[iRoo]), linewidth=0.25)
        
        ax.set_xlabel('time [days]')
        ax.set_ylabel('control signal [$1$]')

        ax.set_xlim([min(t)+5, min(t)+12])
        ax.set_ylim([-0.05, 1.05])

 
        ax.legend(loc='center right', ncol=1)

        configure_axes(ax)

    return plt

# Create the plots
fig = plot_vav(r_base)
save_plot(fig, "vav_base")
fig = plot_vav(r_g36)
save_plot(fig, "vav_g36")

## AHU temperatures

In [6]:
def plot_ahu_temp(readers):
    ''' Main method that plots the results
    '''
    # ------------------------------------------------------
    from buildingspy.io.outputfile import Reader
    import matplotlib.pyplot as plt

    room_styles = ['k', 'b', 'g', 'r', 'c', 'g']
    room_names = ['nor', 'wes', 'sou', 'eas', 'cor']  
    plt.clf()   
    time_scale=86400.
    # ------------------------------------------------------

    # Read results
    for i in [0, 1, 2]:
        (t, TOut) = readers[i].values("weaBus.TDryBul")
        t = t/time_scale
        
        reader=readers[i]
        
        TSup = reader.values("TSup.T")[1]
        TMix = reader.values("TMix.T")[1]
        TRet = reader.values("TRet.T")[1]
  
        # Plot figure
        #plt.subplot(2, 1, 1)
        ax = plt.subplot(3, 1, i+1)

        ax.plot(t, TOut-273.15,    'gray', label='$T_{out}$', linewidth=0.25)
        ax.plot(t, TSup-273.15,    'r', label='$T_{sup}$', linewidth=0.25)
        ax.plot(t, TMix-273.15,    'g', label='$T_{mix}$', linewidth=0.25)
        ax.plot(t, TRet-273.15,    'b', label='$T_{ret}$', linewidth=0.25)

        
        ax.set_xlabel('time [days]')
        ax.set_ylabel('temperature [$^\circ$C]')
    #ax.xaxis.set_ticks(np.arange(min(t)+0, 365, 1))

        ax.set_xlim([min(t)+5, min(t)+12])
 
        ax.legend(loc='center right', ncol=1)

        configure_axes(ax)

    return plt

# Create the plots
fig = plot_ahu_temp(r_base)
save_plot(fig, "TAHU_base")
fig = plot_ahu_temp(r_g36)
save_plot(fig, "TAHU_g36")

## Flow rates and fan speed

In [7]:
def plot_flow_signals(readers):
    ''' Main method that plots the results
    '''
    # ------------------------------------------------------
    from buildingspy.io.outputfile import Reader
    import matplotlib.pyplot as plt

    room_styles = ['k', 'b', 'g', 'r', 'c', 'g']
    room_names = ['nor', 'wes', 'sou', 'eas', 'cor']  
    plt.clf()   
    time_scale=86400.
    # ------------------------------------------------------

    # Read results
    for i in [0, 1, 2]:
        (t, TOut) = readers[i].values("weaBus.TDryBul")
        t = t/time_scale
        
        reader=readers[i]
        
        yFanSup = reader.values("fanSup.y")[1]
        yFanRet = reader.values("fanRet.y")[1]
        yEco = reader.values("eco.y")[1]
  
        # Plot figure
        #plt.subplot(2, 1, 1)
        ax = plt.subplot(3, 1, i+1)

        ax.plot(t, yFanSup,    'r', label='$y_{fan,sup}$', linewidth=0.25)
        ax.plot(t, yFanRet,    'g', label='$y_{fan,ret}$', linewidth=0.25)
        ax.plot(t, yEco,       'b', label='$y_{eco,out}$', linewidth=0.25)

        
        ax.set_xlabel('time [days]')
        ax.set_ylabel('control signal [$1$]')
    #ax.xaxis.set_ticks(np.arange(min(t)+0, 365, 1))

        ax.set_xlim([min(t)+5, min(t)+12])
 
        ax.legend(loc='center right', ncol=1)

        configure_axes(ax)

    return plt

# Create the plots
fig = plot_flow_signals(r_base)
save_plot(fig, "flow_signals_base")
fig = plot_flow_signals(r_g36)
save_plot(fig, "flow_signals_g36")

## Normalized flow rates

In [8]:
def plot_normalized_flow_rates(readers):
    ''' Main method that plots the results
    '''
    # ------------------------------------------------------
    from buildingspy.io.outputfile import Reader
    import matplotlib.pyplot as plt

    room_styles = ['k', 'b', 'g', 'r', 'c', 'g']
    room_names = ['nor', 'wes', 'sou', 'eas', 'cor']  
    plt.clf()   
    time_scale=86400.
    # ------------------------------------------------------

    # Read results
    for i in [0, 1, 2]:
        (t, TOut) = readers[i].values("weaBus.TDryBul")
        t = t/time_scale
        
        reader=readers[i]
        
        mFanSup = reader.values("fanSup.m_flow")[1]/ \
            reader.values("fanSup.m_flow_nominal")[1][1]
        mFanRet = reader.values("fanRet.m_flow")[1]/ \
            reader.values("fanRet.m_flow_nominal")[1][1]
        mEco = reader.values("eco.port_Out.m_flow")[1]/ \
            reader.values("eco.m_flow_nominal")[1][1]
  
        # Plot figure
        #plt.subplot(2, 1, 1)
        ax = plt.subplot(3, 1, i+1)

        ax.plot(t, mFanSup,    'r', label='$\dot m_{fan,sup}/\dot m_{0}$', linewidth=0.25)
        ax.plot(t, mFanRet,    'g', label='$\dot m_{fan,ret}/\dot m_{0}$', linewidth=0.25)
        ax.plot(t, mEco,       'b', label='$\dot m_{eco,out}/\dot m_{eco,0}$', linewidth=0.25)

        
        ax.set_xlabel('time [days]')
        ax.set_ylabel('normalized flow rates [$1$]')
    #ax.xaxis.set_ticks(np.arange(min(t)+0, 365, 1))

        ax.set_xlim([min(t)+5, min(t)+12])
        ax.set_ylim([0, 1])

 
        ax.legend(loc='center right', ncol=1)

        configure_axes(ax)

    return plt

# Create the plots
fig = plot_normalized_flow_rates(r_base)
save_plot(fig, "normalized_flow_base")
fig = plot_normalized_flow_rates(r_g36)
save_plot(fig, "normalized_flow_g36")

# Energy consumption

In [9]:
# Free up storage, then read new data
r_base = list()
r_g36 = list()
r_g36_annual=Reader(cases.get_result_file_name('annual_g36'), "dymola")
print("Read annual G36")
r_base_annual=Reader(cases.get_result_file_name('annual_base'), "dymola")
print("Read annual base")

Read annual G36
Read annual base


In [29]:
def plot_energy(r_base, r_g36):
    from buildingspy.io.outputfile import Reader
    import matplotlib.pyplot as plt
    
    plt.clf()    

    # Conversion to kWh/m2
    conv = 1/3600./1000.
    readers = [r_base, r_g36]
    width = 0.5       # the width of the bars: can also be len(x) sequence

    hea    = [0., 0.]
    cooSen = [0., 0.]
    cooLat = [0., 0.]
    fan    = [0., 0.]
    cooLatBas = [0., 0.]
    fanBas = [0., 0.]
    
    idx = [0, 1]
    for i in idx:
        hea[i]    =  readers[i].values('res.EHea')[1][-1] * conv
        cooSen[i] = -readers[i].values('res.ECooSen')[1][-1] * conv
        cooLat[i] = -readers[i].values('res.ECooLat')[1][-1] * conv
        fan[i]    =  readers[i].values('res.EFan')[1][-1] * conv
        cooLatBas[i] = hea[i] + cooSen[i]
        fanBas[i]    = cooLatBas[i] + cooLat[i]
        
    p1 = plt.bar(idx, hea, width, color='r')
    p2 = plt.bar(idx, cooSen, width, bottom=hea, color='g')
    p3 = plt.bar(idx, cooLat, width, bottom=cooLatBas, color='b')
    p4 = plt.bar(idx, fan,    width, bottom=fanBas, color='k')
        
    plt.ylabel('energy use [kWh/m2]')
    plt.xticks([0, 1], ('base case', 'guideline 36'))
    plt.tick_params(axis=u'x', which=u'both',length=0)

    #plt.yticks(np.arange(0, 81, 10))
    plt.legend((p1[0], p2[0], p3[0], p4[0]), \
               ('heating', 'sensible cooling', 'latent cooling', 'fan'), \
              loc='upper right')
    
    
    save_plot(plt, "energy")
plot_energy(r_base_annual, r_g36_annual)

## Diagnostics

In [11]:
print("Average outdoor air intake, base {} kg/s".format(r_base_annual.mean('eco.port_Out.m_flow')))
print("Average outdoor air intake, G36  {} kg/s".format(r_g36_annual.mean( 'eco.port_Out.m_flow')))

print("CPUtime, base {} h".format(r_base_annual.values('CPUtime')[1][-1]/3600.))
print("CPUtime, G36  {} h".format(r_g36_annual.values( 'CPUtime')[1][-1]/3600.))

Average outdoor air intake, base 0.890886420884 kg/s
Average outdoor air intake, G36  0.754161224939 kg/s
CPUtime, base 1.11004903158 h
CPUtime, G36  1.77151421441 h


In [12]:
(t, y) = r_base_annual.values('flo.sou.air.vol.T')
plt.clf()
plt=Plotter.boxplot(t=t, y=y-273.15, increment=3600, nIncrement=24)

# Decorate, save and show the plot
plt.xlabel('Time [h]') 
plt.ylabel(u'Room temperature [$^\circ$C]') 
plt.grid()
plt.savefig("roomTemperatures.png")
plt.savefig("roomTemperatures.pdf")
plt.title('South zone, base case')
#plt.show() 

Text(0.5,1,u'South zone, base case')

## Energy use